In [27]:
import os
import re
import json
import base64
import tiktoken
import time
import fitz  # PyMuPDF
import pandas as pd
import openai
from IPython.display import Image
from IPython.core.display import HTML
from tqdm.notebook import tqdm
# from openai import OpenAI
import anthropic
import google.generativeai as genai
import PIL
import PIL
from anthropic import HUMAN_PROMPT, AI_PROMPT, Anthropic


In [28]:
MODEL_LIMITS = {
    "gpt-3.5-turbo-0125": 16_385,
    "gpt-4-turbo-2024-04-09": 128_000,
    "gpt-4o-2024-05-13": 128_000,
    "gpt-4o-mini-2024-07-18": 128_000,
    "claude-3-5-sonnet-20240620": 200_000,
    "gemini-1.5-pro-exp-0801": 1000_000,
}

# The cost per token for each model input.
MODEL_COST_PER_INPUT = {
    "gpt-3.5-turbo-0125": 0.0000005,
    "gpt-4-turbo-2024-04-09": 0.00001,
    "gpt-4o-2024-05-13": 0.000005,
    "gpt-4o-mini-2024-07-18": 0.00000015,
    "claude-3-5-sonnet-20240620": 0.000003,
}

# The cost per token for each model output.
MODEL_COST_PER_OUTPUT = {
    "gpt-3.5-turbo-0125": 0.0000015,
    "gpt-4-turbo-2024-04-09": 0.00003,
    "gpt-4o-2024-05-13": 0.000015,
    "gpt-4o-mini-2024-07-18": 0.0000006,
    "claude-3-5-sonnet-20240620":0.000015,
}



In [29]:
# If the question is a multi-choice question and you are unsure which one is correct, you must guess an option.  Please don't ask me any questions and give me the answer in the response.

def call_gemini(text, image, model, files=None):
    system_messages = "You are a data analyst. I will give  you a background introduction and data analysis question. You must give the answer for the input question."
    try:
        if image:
            # organ = PIL.Image.open(image)
            organ = Image(image)
            # inputs = [system_messages +"\n"+ text, organ] + files
            inputs = [system_messages +"\n"+ text, organ]
            response = model.generate_content(inputs)                   
        else:
            # inputs =  [system_messages +"\n"+ text] + files
            inputs =  system_messages +"\n"+ text
            response = model.generate_content(inputs)
            
        return response
    except Exception as e:
        print(e)
        time.sleep(5)
        return e

In [30]:
def gpt_tokenize(string: str, encoding) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoding.encode(string))
    return num_tokens

def claude_tokenize(string: str, api) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = api.count_tokens(string)
    return num_tokens


def find_jpg_files(directory):
    jpg_files = [file for file in os.listdir(directory) if file.lower().endswith('.jpg') or file.lower().endswith('.png')]
    return jpg_files if jpg_files else None

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


def find_excel_files(directory):
    jpg_files = [file for file in os.listdir(directory) if (file.lower().endswith('xlsx') or file.lower().endswith('xlsb') or file.lower().endswith('xlsm')) and not "answer" in file.lower()]
    return jpg_files if jpg_files else None

def read_excel(file_path):
    # 读取Excel文件中的所有sheet
    xls = pd.ExcelFile(file_path)
    sheets = {}
    for sheet_name in xls.sheet_names:
        sheets[sheet_name] = xls.parse(sheet_name)
    return sheets

def dataframe_to_text(df):
    # 将DataFrame转换为文本
    text = df.to_string(index=False)
    return text

def combine_sheets_text(sheets):
    # 将所有sheet的文本内容组合起来
    combined_text = ""
    for sheet_name, df in sheets.items():
        sheet_text = dataframe_to_text(df)
        combined_text += f"Sheet name: {sheet_name}\n{sheet_text}\n\n"
    return combined_text

def read_txt(path):
    with open(path, "r") as f:
        return f.read()

def truncate_text(text, max_tokens=128000):
    # 计算当前文本的token数
    tokens = text.split()
    if len(tokens) > max_tokens:
        # 截断文本以确保不超过最大token数
        text = ' '.join(tokens[-max_tokens:])
    return text

In [47]:
samples = []
with open("./data_.json", "r") as f:
    for line in f:
        samples.append(eval(line.strip()))
len(samples)

43

In [110]:

api_key = "your-api-key"
genai.configure(api_key=api_key)
model = 'gemini-1.5-pro-exp-0801'
client = genai.GenerativeModel(model)
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('Xenova/claude-tokenizer')


In [111]:
a= client.generate_content("hello")

I0000 00:00:1723842519.491469 1558550 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [112]:
a

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Hello! How can I help you today?\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
  

In [113]:
response = call_gemini("你好",None, client, [])

  0%|          | 0/2 [00:00<?, ?it/s]

existing 33 answers


  0%|          | 0/17 [00:00<?, ?it/s]

Total cost:  3.255e-05
Total cost:  8.295e-05
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 2: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.000112
Total cost:  0.00014209999999999998
Total cost:  0.0001757
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 2: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.00020789999999999998
Total cost:  0.00024045
Total cost:  0.0002779
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 2: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.00031114999999999995
Total cost:  0.0003401

  0%|          | 0/20 [00:00<?, ?it/s]

429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 2: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.0009544499999999998
Total cost:  0.0013303499999999997
Total cost:  0.0018168499999999996
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.0024793999999999997
Total cost:  0.0027975499999999998
Total cost:  0.0031702999999999996
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 2: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.0034201999999999995
Total cost:  0.0037502499999999997
Total cost:  0.00404355
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' objec

In [24]:
response

google.api_core.exceptions.ResourceExhausted('Resource has been exhausted (e.g. check quota).')

In [25]:
response

google.api_core.exceptions.ResourceExhausted('Resource has been exhausted (e.g. check quota).')

In [51]:


data_path = './data'

total_cost = 0

for id in tqdm(range(0, len(samples))):
    # print(sample)
    sample =samples[id]
    if len(sample["questions"]) > 0:
        start = sample["questions"][0]
        end = sample["questions"][-1]
        # print(start)
        # print(end)
        image = find_jpg_files(os.path.join(data_path, sample["id"]))
        if image:
            image = os.path.join(data_path, sample["id"], image[0])
        
        excel_content = ""
        excels = find_excel_files(os.path.join(data_path, sample["id"]))
        # files = []
        
        if excels:
            for excel in excels:
                excel_file_path = os.path.join(data_path,  sample["id"], excel)
                sheets = read_excel(excel_file_path)
                combined_text = combine_sheets_text(sheets)
                excel_content += f"The excel file {excel} is: " + combined_text
                # files.append(genai.upload_file(excel_file_path))

        introduction = read_txt(os.path.join(data_path, sample["id"], "introduction.txt"))
        questions = []
        for question_name in sample["questions"]:
            questions.append(read_txt(os.path.join(data_path, sample["id"], question_name+".txt")))
            
        # print(workbooks)
        
        text = ""
        if excel_content:
            text += f"The workbook is detailed as follows. {excel_content} \n"
        text += f"The introduction is detailed as follows. \n {introduction} \n"
        answers = []
        
        line_num = 0
        save_path = os.path.join("./evaluation/save_process", model)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        if os.path.exists(os.path.join(save_path, sample['id']+".json")):
            with open(os.path.join(save_path, sample['id']+".json"), "r") as f:
                for line in f:
                    line_num += 1
        print(f"existing {line_num} answers")
        save_f =  open(os.path.join(save_path, sample['id']+".json"), "a+")
        
        for question in tqdm(questions[line_num:]):
            prompt = text +  f"The questions are detailed as follows. \n {question}"
        
            # print(len(encoding.encode(prompt)))
            cut_text = tokenizer.decode(tokenizer.encode(prompt)[6000-MODEL_LIMITS[model]:])
            # print(len(encoding.encode(prompt)))
            # print(prompt)
            # text = truncate_text(text, 20000)
            error_num = 0
            while True:
                try:
                    start = time.time()
                    # response = call_gemini(cut_text, image, client, files)
                    response = call_gemini(cut_text, image, client)
                    
                    cost = 0
                    if response.usage_metadata.prompt_token_count >128000:
                        cost += response.usage_metadata.prompt_token_count * 0.0000007
                    else:
                        cost += 0.00000035 * response.usage_metadata.prompt_token_count
                        
                    if response.usage_metadata.candidates_token_count > 128000:
                        cost += 0.0000021 * response.usage_metadata.candidates_token_count
                    else:
                        cost += 0.00000105 * response.usage_metadata.candidates_token_count
                    # cost = response.usage_metadata.prompt_token_count * MODEL_COST_PER_OUTPUT[model] + response.usage.input_tokens * MODEL_COST_PER_INPUT[model]
                    ans = {"id": sample["id"], "model": model, "input": response.usage_metadata.prompt_token_count,
                                    "output": response.usage_metadata.candidates_token_count, "cost": cost, "time": time.time()-start, "response": response.text}
                    
                    answers.append({"id": sample["id"], "model": model, "input": response.usage_metadata.prompt_token_count,
                                    "output": response.usage_metadata.candidates_token_count, "cost": cost, "time": time.time()-start, "response": response.text})
                    total_cost += cost
                    print("Total cost: ", total_cost)
                    json.dump(ans, save_f)
                    save_f.write("\n")
                    save_f.flush()
                    break
                except Exception as e:
                    error_num += 1
                    print(f"error {error_num}: {e}")
                    
                    time.sleep(10)
        
            # time.sleep(58)
            
            # break
        save_f.close()

            

  0%|          | 0/39 [00:00<?, ?it/s]

existing 10 answers


0it [00:00, ?it/s]

existing 10 answers


0it [00:00, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

Total cost:  0.25977279999999997
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.26281465
Total cost:  0.2657305
Total cost:  0.26851159999999996
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 2: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.27160524999999996
Total cost:  0.27452845
Total cost:  0.27714049999999996
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.28002414999999997
Total cost:  0.2826026
Total cost:  0.2851919
existing 0 answers


  0%|          | 0/10 [00:00<?, ?it/s]

429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 2: 'ResourceExhausted' object has no attribute 'usage_metadata'
Total cost:  0.28850465
Total cost:  0.29198329999999995
Total cost:  0.29585079999999997
Total cost:  0.299355
429 Resource has been exhausted (e.g. check quota).
error 1: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 2: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 3: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 4: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e.g. check quota).
error 5: 'ResourceExhausted' object has no attribute 'usage_metadata'
429 Resource has been exhausted (e

KeyboardInterrupt: 

In [86]:
response

google.api_core.exceptions.ResourceExhausted('Resource has been exhausted (e.g. check quota).')

In [84]:
response.text

'* **Days in Q4 2023:** October (31 days) + November (30 days) + December (31 days) = 92 days\n* **Shutdown in Q4 2023:** From 25th October to 10th November = 16 days\n* **Operational Days:** 92 days - 16 days = 76 days\n\n**Answer: E. 76** \n'

In [ ]:
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.choices[0].message.content)
print(response.model)

print(response)